## Functions

In [1]:
# %matplotlib inline

import numpy as np
from skimage import transform, io, filters
import skimage.util
import os
import glob
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, BatchNormalization
from keras import layers
import tensorflow as tf
import time
from keras import backend as K
# from tensorflow.keras import layers

# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


def get_unet_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.UpSampling2D(2)(x)

        # Project residual
        residual = layers.UpSampling2D(2)(previous_block_activation)
        residual = layers.Conv2D(filters, 1, padding="same")(residual)
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)
    return model

def path_planning_model():
    x = Input(shape=(None, None, 1))

    net = Conv2D(filters=64, kernel_size=[3, 3], strides=[1, 1], padding="same", kernel_initializer='orthogonal', activation='relu')(x)
    net = BatchNormalization()(net)
    for i in range(19):
        net = Conv2D(filters=64, kernel_size=[3, 3], strides=[1, 1], padding="same", kernel_initializer='orthogonal', activation='relu')(net)
        net = BatchNormalization()(net)
        
    net = Conv2D(filters=1, kernel_size=[3, 3], strides=[1, 1], padding="same", kernel_initializer='orthogonal', activation='sigmoid')(net)
    net = BatchNormalization()(net)
    # net = Conv2D(filters=1, kernel_size=[3, 3], strides=[1, 1], padding="same", kernel_initializer='orthogonal', activation='sigmoid')(net)
    # net = BatchNormalization()(net)	
    net = Dropout(0.10)(net)
    # Define the model
    model = keras.Model(inputs=x, outputs=net)
    return model

# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred.flatten(), 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    b = plt.plot(t['pred'].tolist(),label='prediction',color = 'orange')
    a = plt.plot(t['y'].tolist(),label='expected',color = 'blue')
    plt.ylabel('output')
    plt.legend()
    plt.show()

Using TensorFlow backend.
/home/chris/anaconda3/envs/AI4AVenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/chris/anaconda3/envs/AI4AVenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/chris/anaconda3/envs/AI4AVenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/chris/anaco

## Data

In [2]:
# Base data directories
root_dir = ".."
data_dir = os.path.join(root_dir,"data")

bugtrap_forest_dir = os.path.join(data_dir, "bugtrap_forest" )
forest_dir = os.path.join(data_dir, "forest" )
multiple_bugraps_dir = os.path.join(data_dir, "multiple_bugtraps" )
shifting_gaps_dir = os.path.join(data_dir, "shifting_gaps" )

## Mazes

In [3]:
# Mazes
mazes_dir = os.path.join(data_dir, "mazes" )
mazes_train_csv_path = os.path.join(mazes_dir,"mazes_train.csv")
mazes_test_csv_path = os.path.join(mazes_dir,"mazes_test.csv")
mazes_valid_csv_path = os.path.join(mazes_dir,"mazes_validation.csv")
mazes_train_data = pd.read_csv(mazes_train_csv_path)
mazes_test_data = pd.read_csv(mazes_test_csv_path)
mazes_valid_data = pd.read_csv(mazes_valid_csv_path)

### Train Data

In [4]:
# X - Map image file
# Y - Path image file

x_train_imgs = glob.glob(os.path.join(mazes_dir, "train/*.png"))
x_imgs_train = []
x_labels_train_mazes = []
for img_path in x_train_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_train_mazes.append(img_name)
print("X_train_mazes")
print(f"# of images: {len(x_imgs_train)}")
X_train_mazes = np.array(x_imgs_train, dtype='float32')
X_train_mazes/=255
print(f"{X_train_mazes.ndim}")
print(f"{X_train_mazes.shape}")
# print(f"{X_train_mazes.dtype.names}")
# print(f"{X_train_mazes[0]}")

y_train_imgs = glob.glob(os.path.join(mazes_dir, "train_path/*.png"))
y_imgs_train = []
y_labels_train_mazes = []
for img_path in y_train_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_train_mazes.append(img_name)
print("y_train_mazes")
print(f"# of images: {len(y_imgs_train)}")
y_train_mazes = np.array(y_imgs_train, dtype='float32')
y_train_mazes/=255
print(f"{y_train_mazes.ndim}")
print(f"{y_train_mazes.shape}")
# print(f"{y_train_mazes.dtype.names}")
# print(f"{y_train_mazes[0]}")

X_train_mazes
# of images: 800
3
(800, 32, 32)
y_train_mazes
# of images: 800
3
(800, 32, 32)


### Validation Data

In [5]:
x_valid_imgs = glob.glob(os.path.join(mazes_dir, "validation/*.png"))
x_imgs_valid = []
x_labels_valid_mazes = []
for img_path in x_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_valid_mazes.append(img_name)
print("X_valid_mazes")
print(f"# of images: {len(x_imgs_valid)}")
X_valid_mazes = np.array(x_imgs_valid, dtype='float32')
X_valid_mazes/=255
print(f"{X_valid_mazes.ndim}")
print(f"{X_valid_mazes.shape}")
# print(f"{X_valid_mazes.dtype.names}")
# print(f"{X_valid_mazes[0]}")

y_valid_imgs = glob.glob(os.path.join(mazes_dir, "validation_path/*.png"))
y_imgs_valid = []
y_labels_valid_mazes = []
for img_path in y_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_valid_mazes.append(img_name)
print("y_valid_mazes")
print(f"# of images: {len(y_imgs_valid)}")
y_valid_mazes = np.array(y_imgs_valid, dtype='float32')
y_valid_mazes/=255
print(f"{y_valid_mazes.ndim}")
print(f"{y_valid_mazes.shape}")
# print(f"{y_valid_mazes.dtype.names}")
# print(f"{y_valid_mazes[0]}")

X_valid_mazes
# of images: 100
3
(100, 32, 32)
y_valid_mazes
# of images: 100
3
(100, 32, 32)


### Test Data

In [6]:
x_test_imgs = glob.glob(os.path.join(mazes_dir, "test/*.png"))
imgs_test = []
x_labels_test_mazes = []
for img_path in x_test_imgs:
    img = io.imread(img_path, as_gray=True)
    imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_test_mazes.append(img_name)
print("X_test_mazes")
print(f"# of images: {len(imgs_test)}")
X_test_mazes = np.array(imgs_test, dtype='float32')
X_test_mazes/=255
# X_test = np.array(imgs_test)
print(f"{X_test_mazes.ndim}")
print(f"{X_test_mazes.shape}")
# print(f"{X_test_mazes.dtype.names}")
# print(f"{X_test_mazes[0]}")

y_test_imgs = glob.glob(os.path.join(mazes_dir, "test_path/*.png"))
y_imgs_test = []
y_labels_test_mazes = []
for img_path in y_test_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_test_mazes.append(img_name)
print("y_test_mazes")
print(f"# of images: {len(y_imgs_test)}")
y_test_mazes = np.array(y_imgs_test, dtype='float32')
y_test_mazes/=255
print(f"{y_test_mazes.ndim}")
print(f"{y_test_mazes.shape}")
# print(f"{y_test_mazes.dtype.names}")
# print(f"{y_test_mazes[0]}")

X_test_mazes
# of images: 100
3
(100, 32, 32)
y_test_mazes
# of images: 100
3
(100, 32, 32)


In [7]:
# input_shape = (32, 32, 1)
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    X_train_mazes = X_train_mazes.reshape(X_train_mazes.shape[0], 1, img_rows, img_cols)
    X_valid_mazes = X_valid_mazes.reshape(X_valid_mazes.shape[0], 1, img_rows, img_cols)
    X_test_mazes = X_test_mazes.reshape(X_test_mazes.shape[0], 1, img_rows, img_cols)
    y_train_mazes = y_train_mazes.reshape(y_train_mazes.shape[0], 1, img_rows, img_cols)
    y_valid_mazes = y_valid_mazes.reshape(y_valid_mazes.shape[0], 1, img_rows, img_cols)
    y_test_mazes = y_test_mazes.reshape(y_test_mazes.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train_mazes = X_train_mazes.reshape(X_train_mazes.shape[0], img_rows, img_cols, 1)
    X_valid_mazes = X_valid_mazes.reshape(X_valid_mazes.shape[0], img_rows, img_cols, 1)
    X_test_mazes = X_test_mazes.reshape(X_test_mazes.shape[0], img_rows, img_cols, 1)
    y_train_mazes = y_train_mazes.reshape(y_train_mazes.shape[0], img_rows, img_cols, 1)
    y_valid_mazes = y_valid_mazes.reshape(y_valid_mazes.shape[0], img_rows, img_cols, 1)
    y_test_mazes = y_test_mazes.reshape(y_test_mazes.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

## Alternating Gaps

In [8]:
# alternating_gaps
alternating_gaps_dir = os.path.join(data_dir, "alternating_gaps" )
alternating_gaps_train_csv_path = os.path.join(alternating_gaps_dir,"alternating_gaps_train.csv")
alternating_gaps_test_csv_path = os.path.join(alternating_gaps_dir,"alternating_gaps_test.csv")
alternating_gaps_valid_csv_path = os.path.join(alternating_gaps_dir,"alternating_gaps_validation.csv")
alternating_gaps_train_data = pd.read_csv(alternating_gaps_train_csv_path)
alternating_gaps_test_data = pd.read_csv(alternating_gaps_test_csv_path)
alternating_gaps_valid_data = pd.read_csv(alternating_gaps_valid_csv_path)

### Train Data

In [9]:
# X - Map image file
# Y - Path image file

x_train_imgs = glob.glob(os.path.join(alternating_gaps_dir, "train/*.png"))
x_imgs_train = []
x_labels_train_alternating_gaps = []
for img_path in x_train_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_train_alternating_gaps.append(img_name)
print("X_train_alternating_gaps")
print(f"# of images: {len(x_imgs_train)}")
X_train_alternating_gaps = np.array(x_imgs_train, dtype='float32')
X_train_alternating_gaps/=255
print(f"{X_train_alternating_gaps.ndim}")
print(f"{X_train_alternating_gaps.shape}")
# print(f"{X_train_alternating_gaps.dtype.names}")
# print(f"{X_train_alternating_gaps[0]}")

y_train_imgs = glob.glob(os.path.join(alternating_gaps_dir, "train_path/*.png"))
y_imgs_train = []
y_labels_train_alternating_gaps = []
for img_path in y_train_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_train_alternating_gaps.append(img_name)
print("y_train_alternating_gaps")
print(f"# of images: {len(y_imgs_train)}")
y_train_alternating_gaps = np.array(y_imgs_train, dtype='float32')
y_train_alternating_gaps/=255
print(f"{y_train_alternating_gaps.ndim}")
print(f"{y_train_alternating_gaps.shape}")
# print(f"{y_train_alternating_gaps.dtype.names}")
# print(f"{y_train_alternating_gaps[0]}")

X_train_alternating_gaps
# of images: 800
3
(800, 32, 32)
y_train_alternating_gaps
# of images: 800
3
(800, 32, 32)


### Validation Data

In [10]:
x_valid_imgs = glob.glob(os.path.join(alternating_gaps_dir, "validation/*.png"))
x_imgs_valid = []
x_labels_valid_alternating_gaps = []
for img_path in x_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_valid_alternating_gaps.append(img_name)
print("X_valid_alternating_gaps")
print(f"# of images: {len(x_imgs_valid)}")
X_valid_alternating_gaps = np.array(x_imgs_valid, dtype='float32')
X_valid_alternating_gaps/=255
print(f"{X_valid_alternating_gaps.ndim}")
print(f"{X_valid_alternating_gaps.shape}")
# print(f"{X_valid_alternating_gaps.dtype.names}")
# print(f"{X_valid_alternating_gaps[0]}")

y_valid_imgs = glob.glob(os.path.join(alternating_gaps_dir, "validation_path/*.png"))
y_imgs_valid = []
y_labels_valid_alternating_gaps = []
for img_path in y_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_valid_alternating_gaps.append(img_name)
print("y_valid_alternating_gaps")
print(f"# of images: {len(y_imgs_valid)}")
y_valid_alternating_gaps = np.array(y_imgs_valid, dtype='float32')
y_valid_alternating_gaps/=255
print(f"{y_valid_alternating_gaps.ndim}")
print(f"{y_valid_alternating_gaps.shape}")
# print(f"{y_valid_alternating_gaps.dtype.names}")
# print(f"{y_valid_alternating_gaps[0]}")

X_valid_alternating_gaps
# of images: 100
3
(100, 32, 32)
y_valid_alternating_gaps
# of images: 100
3
(100, 32, 32)


### Test Data

In [11]:
x_test_imgs = glob.glob(os.path.join(alternating_gaps_dir, "test/*.png"))
imgs_test = []
x_labels_test_alternating_gaps = []
for img_path in x_test_imgs:
    img = io.imread(img_path, as_gray=True)
    imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_test_alternating_gaps.append(img_name)
print("X_test_alternating_gaps")
print(f"# of images: {len(imgs_test)}")
X_test_alternating_gaps = np.array(imgs_test, dtype='float32')
X_test_alternating_gaps/=255
# X_test = np.array(imgs_test)
print(f"{X_test_alternating_gaps.ndim}")
print(f"{X_test_alternating_gaps.shape}")
# print(f"{X_test_alternating_gaps.dtype.names}")
# print(f"{X_test_alternating_gaps[0]}")

y_test_imgs = glob.glob(os.path.join(alternating_gaps_dir, "test_path/*.png"))
y_imgs_test = []
y_labels_test_alternating_gaps = []
for img_path in y_test_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_test_alternating_gaps.append(img_name)
print("y_test_alternating_gaps")
print(f"# of images: {len(y_imgs_test)}")
y_test_alternating_gaps = np.array(y_imgs_test, dtype='float32')
y_test_alternating_gaps/=255
print(f"{y_test_alternating_gaps.ndim}")
print(f"{y_test_alternating_gaps.shape}")
# print(f"{y_test_alternating_gaps.dtype.names}")
# print(f"{y_test_alternating_gaps[0]}")

X_test_alternating_gaps
# of images: 100
3
(100, 32, 32)
y_test_alternating_gaps
# of images: 100
3
(100, 32, 32)


In [12]:
# input_shape = (32, 32, 1)
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    X_train_alternating_gaps = X_train_alternating_gaps.reshape(X_train_alternating_gaps.shape[0], 1, img_rows, img_cols)
    X_valid_alternating_gaps = X_valid_alternating_gaps.reshape(X_valid_alternating_gaps.shape[0], 1, img_rows, img_cols)
    X_test_alternating_gaps = X_test_alternating_gaps.reshape(X_test_alternating_gaps.shape[0], 1, img_rows, img_cols)
    y_train_alternating_gaps = y_train_alternating_gaps.reshape(y_train_alternating_gaps.shape[0], 1, img_rows, img_cols)
    y_valid_alternating_gaps = y_valid_alternating_gaps.reshape(y_valid_alternating_gaps.shape[0], 1, img_rows, img_cols)
    y_test_alternating_gaps = y_test_alternating_gaps.reshape(y_test_alternating_gaps.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train_alternating_gaps = X_train_alternating_gaps.reshape(X_train_alternating_gaps.shape[0], img_rows, img_cols, 1)
    X_valid_alternating_gaps = X_valid_alternating_gaps.reshape(X_valid_alternating_gaps.shape[0], img_rows, img_cols, 1)
    X_test_alternating_gaps = X_test_alternating_gaps.reshape(X_test_alternating_gaps.shape[0], img_rows, img_cols, 1)
    y_train_alternating_gaps = y_train_alternating_gaps.reshape(y_train_alternating_gaps.shape[0], img_rows, img_cols, 1)
    y_valid_alternating_gaps = y_valid_alternating_gaps.reshape(y_valid_alternating_gaps.shape[0], img_rows, img_cols, 1)
    y_test_alternating_gaps = y_test_alternating_gaps.reshape(y_test_alternating_gaps.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

## Bugtrap Forest

In [13]:
# Bugtrap Forest
bugtrap_forest_dir = os.path.join(data_dir, "bugtrap_forest" )
bugtrap_forest_train_csv_path = os.path.join(bugtrap_forest_dir,"bugtrap_forest_train.csv")
bugtrap_forest_test_csv_path = os.path.join(bugtrap_forest_dir,"bugtrap_forest_test.csv")
bugtrap_forest_valid_csv_path = os.path.join(bugtrap_forest_dir,"bugtrap_forest_validation.csv")
bugtrap_forest_train_data = pd.read_csv(bugtrap_forest_train_csv_path)
bugtrap_forest_test_data = pd.read_csv(bugtrap_forest_test_csv_path)
bugtrap_forest_valid_data = pd.read_csv(bugtrap_forest_valid_csv_path)

### Train Data

In [14]:
# X - Map image file
# Y - Path image file

x_train_imgs = glob.glob(os.path.join(bugtrap_forest_dir, "train/*.png"))
x_imgs_train = []
x_labels_train_bugtrap_forest = []
for img_path in x_train_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_train_bugtrap_forest.append(img_name)
print("X_train_bugtrap_forest")
print(f"# of images: {len(x_imgs_train)}")
X_train_bugtrap_forest = np.array(x_imgs_train, dtype='float32')
X_train_bugtrap_forest/=255
print(f"{X_train_bugtrap_forest.ndim}")
print(f"{X_train_bugtrap_forest.shape}")
# print(f"{X_train_bugtrap_forest.dtype.names}")
# print(f"{X_train_bugtrap_forest[0]}")

y_train_imgs = glob.glob(os.path.join(bugtrap_forest_dir, "train_path/*.png"))
y_imgs_train = []
y_labels_train_bugtrap_forest = []
for img_path in y_train_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_train_bugtrap_forest.append(img_name)
print("y_train_bugtrap_forest")
print(f"# of images: {len(y_imgs_train)}")
y_train_bugtrap_forest = np.array(y_imgs_train, dtype='float32')
y_train_bugtrap_forest/=255
print(f"{y_train_bugtrap_forest.ndim}")
print(f"{y_train_bugtrap_forest.shape}")
# print(f"{y_train_bugtrap_forest.dtype.names}")
# print(f"{y_train_bugtrap_forest[0]}")

X_train_bugtrap_forest
# of images: 800
3
(800, 32, 32)
y_train_bugtrap_forest
# of images: 800
3
(800, 32, 32)


### Validation Data

In [15]:
x_valid_imgs = glob.glob(os.path.join(bugtrap_forest_dir, "validation/*.png"))
x_imgs_valid = []
x_labels_valid_bugtrap_forest = []
for img_path in x_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_valid_bugtrap_forest.append(img_name)
print("X_valid_bugtrap_forest")
print(f"# of images: {len(x_imgs_valid)}")
X_valid_bugtrap_forest = np.array(x_imgs_valid, dtype='float32')
X_valid_bugtrap_forest/=255
print(f"{X_valid_bugtrap_forest.ndim}")
print(f"{X_valid_bugtrap_forest.shape}")
# print(f"{X_valid_bugtrap_forest.dtype.names}")
# print(f"{X_valid_bugtrap_forest[0]}")

y_valid_imgs = glob.glob(os.path.join(bugtrap_forest_dir, "validation_path/*.png"))
y_imgs_valid = []
y_labels_valid_bugtrap_forest = []
for img_path in y_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_valid_bugtrap_forest.append(img_name)
print("y_valid_bugtrap_forest")
print(f"# of images: {len(y_imgs_valid)}")
y_valid_bugtrap_forest = np.array(y_imgs_valid, dtype='float32')
y_valid_bugtrap_forest/=255
print(f"{y_valid_bugtrap_forest.ndim}")
print(f"{y_valid_bugtrap_forest.shape}")
# print(f"{y_valid_bugtrap_forest.dtype.names}")
# print(f"{y_valid_bugtrap_forest[0]}")

X_valid_bugtrap_forest
# of images: 100
3
(100, 32, 32)
y_valid_bugtrap_forest
# of images: 100
3
(100, 32, 32)


### Test Data

In [16]:
x_test_imgs = glob.glob(os.path.join(bugtrap_forest_dir, "test/*.png"))
imgs_test = []
x_labels_test_bugtrap_forest = []
for img_path in x_test_imgs:
    img = io.imread(img_path, as_gray=True)
    imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_test_bugtrap_forest.append(img_name)
print("X_test_bugtrap_forest")
print(f"# of images: {len(imgs_test)}")
X_test_bugtrap_forest = np.array(imgs_test, dtype='float32')
X_test_bugtrap_forest/=255
# X_test = np.array(imgs_test)
print(f"{X_test_bugtrap_forest.ndim}")
print(f"{X_test_bugtrap_forest.shape}")
# print(f"{X_test_bugtrap_forest.dtype.names}")
# print(f"{X_test_bugtrap_forest[0]}")

y_test_imgs = glob.glob(os.path.join(bugtrap_forest_dir, "test_path/*.png"))
y_imgs_test = []
y_labels_test_bugtrap_forest = []
for img_path in y_test_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_test_bugtrap_forest.append(img_name)
print("y_test_bugtrap_forest")
print(f"# of images: {len(y_imgs_test)}")
y_test_bugtrap_forest = np.array(y_imgs_test, dtype='float32')
y_test_bugtrap_forest/=255
print(f"{y_test_bugtrap_forest.ndim}")
print(f"{y_test_bugtrap_forest.shape}")
# print(f"{y_test_bugtrap_forest.dtype.names}")
# print(f"{y_test_bugtrap_forest[0]}")

X_test_bugtrap_forest
# of images: 100
3
(100, 32, 32)
y_test_bugtrap_forest
# of images: 100
3
(100, 32, 32)


In [17]:
# input_shape = (32, 32, 1)
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    X_train_bugtrap_forest = X_train_bugtrap_forest.reshape(X_train_bugtrap_forest.shape[0], 1, img_rows, img_cols)
    X_valid_bugtrap_forest = X_valid_bugtrap_forest.reshape(X_valid_bugtrap_forest.shape[0], 1, img_rows, img_cols)
    X_test_bugtrap_forest = X_test_bugtrap_forest.reshape(X_test_bugtrap_forest.shape[0], 1, img_rows, img_cols)
    y_train_bugtrap_forest = y_train_bugtrap_forest.reshape(y_train_bugtrap_forest.shape[0], 1, img_rows, img_cols)
    y_valid_bugtrap_forest = y_valid_bugtrap_forest.reshape(y_valid_bugtrap_forest.shape[0], 1, img_rows, img_cols)
    y_test_bugtrap_forest = y_test_bugtrap_forest.reshape(y_test_bugtrap_forest.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train_bugtrap_forest = X_train_bugtrap_forest.reshape(X_train_bugtrap_forest.shape[0], img_rows, img_cols, 1)
    X_valid_bugtrap_forest = X_valid_bugtrap_forest.reshape(X_valid_bugtrap_forest.shape[0], img_rows, img_cols, 1)
    X_test_bugtrap_forest = X_test_bugtrap_forest.reshape(X_test_bugtrap_forest.shape[0], img_rows, img_cols, 1)
    y_train_bugtrap_forest = y_train_bugtrap_forest.reshape(y_train_bugtrap_forest.shape[0], img_rows, img_cols, 1)
    y_valid_bugtrap_forest = y_valid_bugtrap_forest.reshape(y_valid_bugtrap_forest.shape[0], img_rows, img_cols, 1)
    y_test_bugtrap_forest = y_test_bugtrap_forest.reshape(y_test_bugtrap_forest.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

## Multiple Bugtraps

In [18]:
# Multiple Bugtraps
multiple_bugtraps_dir = os.path.join(data_dir, "multiple_bugtraps" )
multiple_bugtraps_train_csv_path = os.path.join(multiple_bugtraps_dir,"multiple_bugtraps_train.csv")
multiple_bugtraps_test_csv_path = os.path.join(multiple_bugtraps_dir,"multiple_bugtraps_test.csv")
multiple_bugtraps_valid_csv_path = os.path.join(multiple_bugtraps_dir,"multiple_bugtraps_validation.csv")
multiple_bugtraps_train_data = pd.read_csv(multiple_bugtraps_train_csv_path)
multiple_bugtraps_test_data = pd.read_csv(multiple_bugtraps_test_csv_path)
multiple_bugtraps_valid_data = pd.read_csv(multiple_bugtraps_valid_csv_path)

FileNotFoundError: File b'../data/multiple_bugtraps/mazes_test.csv' does not exist

### Train Data

In [ ]:
# X - Map image file
# Y - Path image file

x_train_imgs = glob.glob(os.path.join(multiple_bugtraps_dir, "train/*.png"))
x_imgs_train = []
x_labels_train_multiple_bugtraps = []
for img_path in x_train_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_train_multiple_bugtraps.append(img_name)
print("X_train_multiple_bugtraps")
print(f"# of images: {len(x_imgs_train)}")
X_train_multiple_bugtraps = np.array(x_imgs_train, dtype='float32')
X_train_multiple_bugtraps/=255
print(f"{X_train_multiple_bugtraps.ndim}")
print(f"{X_train_multiple_bugtraps.shape}")
# print(f"{X_train_multiple_bugtraps.dtype.names}")
# print(f"{X_train_multiple_bugtraps[0]}")

y_train_imgs = glob.glob(os.path.join(multiple_bugtraps_dir, "train_path/*.png"))
y_imgs_train = []
y_labels_train_multiple_bugtraps = []
for img_path in y_train_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_train_multiple_bugtraps.append(img_name)
print("y_train_multiple_bugtraps")
print(f"# of images: {len(y_imgs_train)}")
y_train_multiple_bugtraps = np.array(y_imgs_train, dtype='float32')
y_train_multiple_bugtraps/=255
print(f"{y_train_multiple_bugtraps.ndim}")
print(f"{y_train_multiple_bugtraps.shape}")
# print(f"{y_train_multiple_bugtraps.dtype.names}")
# print(f"{y_train_multiple_bugtraps[0]}")

### Validation Data

In [ ]:
x_valid_imgs = glob.glob(os.path.join(multiple_bugtraps_dir, "validation/*.png"))
x_imgs_valid = []
x_labels_valid_multiple_bugtraps = []
for img_path in x_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_valid_multiple_bugtraps.append(img_name)
print("X_valid_multiple_bugtraps")
print(f"# of images: {len(x_imgs_valid)}")
X_valid_multiple_bugtraps = np.array(x_imgs_valid, dtype='float32')
X_valid_multiple_bugtraps/=255
print(f"{X_valid_multiple_bugtraps.ndim}")
print(f"{X_valid_multiple_bugtraps.shape}")
# print(f"{X_valid_multiple_bugtraps.dtype.names}")
# print(f"{X_valid_multiple_bugtraps[0]}")

y_valid_imgs = glob.glob(os.path.join(multiple_bugtraps_dir, "validation_path/*.png"))
y_imgs_valid = []
y_labels_valid_multiple_bugtraps = []
for img_path in y_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_valid_multiple_bugtraps.append(img_name)
print("y_valid_multiple_bugtraps")
print(f"# of images: {len(y_imgs_valid)}")
y_valid_multiple_bugtraps = np.array(y_imgs_valid, dtype='float32')
y_valid_multiple_bugtraps/=255
print(f"{y_valid_multiple_bugtraps.ndim}")
print(f"{y_valid_multiple_bugtraps.shape}")
# print(f"{y_valid_multiple_bugtraps.dtype.names}")
# print(f"{y_valid_multiple_bugtraps[0]}")

### Test Data

In [ ]:
x_test_imgs = glob.glob(os.path.join(multiple_bugtraps_dir, "test/*.png"))
imgs_test = []
x_labels_test_multiple_bugtraps = []
for img_path in x_test_imgs:
    img = io.imread(img_path, as_gray=True)
    imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_test_multiple_bugtraps.append(img_name)
print("X_test_multiple_bugtraps")
print(f"# of images: {len(imgs_test)}")
X_test_multiple_bugtraps = np.array(imgs_test, dtype='float32')
X_test_multiple_bugtraps/=255
# X_test = np.array(imgs_test)
print(f"{X_test_multiple_bugtraps.ndim}")
print(f"{X_test_multiple_bugtraps.shape}")
# print(f"{X_test_multiple_bugtraps.dtype.names}")
# print(f"{X_test_multiple_bugtraps[0]}")

y_test_imgs = glob.glob(os.path.join(multiple_bugtraps_dir, "test_path/*.png"))
y_imgs_test = []
y_labels_test_multiple_bugtraps = []
for img_path in y_test_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_test_multiple_bugtraps.append(img_name)
print("y_test_multiple_bugtraps")
print(f"# of images: {len(y_imgs_test)}")
y_test_multiple_bugtraps = np.array(y_imgs_test, dtype='float32')
y_test_multiple_bugtraps/=255
print(f"{y_test_multiple_bugtraps.ndim}")
print(f"{y_test_multiple_bugtraps.shape}")
# print(f"{y_test_multiple_bugtraps.dtype.names}")
# print(f"{y_test_multiple_bugtraps[0]}")

In [ ]:
# input_shape = (32, 32, 1)
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    X_train_multiple_bugtraps = X_train_multiple_bugtraps.reshape(X_train_multiple_bugtraps.shape[0], 1, img_rows, img_cols)
    X_valid_multiple_bugtraps = X_valid_multiple_bugtraps.reshape(X_valid_multiple_bugtraps.shape[0], 1, img_rows, img_cols)
    X_test_multiple_bugtraps = X_test_multiple_bugtraps.reshape(X_test_multiple_bugtraps.shape[0], 1, img_rows, img_cols)
    y_train_multiple_bugtraps = y_train_multiple_bugtraps.reshape(y_train_multiple_bugtraps.shape[0], 1, img_rows, img_cols)
    y_valid_multiple_bugtraps = y_valid_multiple_bugtraps.reshape(y_valid_multiple_bugtraps.shape[0], 1, img_rows, img_cols)
    y_test_multiple_bugtraps = y_test_multiple_bugtraps.reshape(y_test_multiple_bugtraps.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train_multiple_bugtraps = X_train_multiple_bugtraps.reshape(X_train_multiple_bugtraps.shape[0], img_rows, img_cols, 1)
    X_valid_multiple_bugtraps = X_valid_multiple_bugtraps.reshape(X_valid_multiple_bugtraps.shape[0], img_rows, img_cols, 1)
    X_test_multiple_bugtraps = X_test_multiple_bugtraps.reshape(X_test_multiple_bugtraps.shape[0], img_rows, img_cols, 1)
    y_train_multiple_bugtraps = y_train_multiple_bugtraps.reshape(y_train_multiple_bugtraps.shape[0], img_rows, img_cols, 1)
    y_valid_multiple_bugtraps = y_valid_multiple_bugtraps.reshape(y_valid_multiple_bugtraps.shape[0], img_rows, img_cols, 1)
    y_test_multiple_bugtraps = y_test_multiple_bugtraps.reshape(y_test_multiple_bugtraps.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

## Shifting Gaps

In [ ]:
# Shifting Gaps
shifting_gaps_dir = os.path.join(data_dir, "shifting_gaps" )
shifting_gaps_train_csv_path = os.path.join(shifting_gaps_dir,"shifting_gaps_train.csv")
shifting_gaps_test_csv_path = os.path.join(shifting_gaps_dir,"shifting_gaps_test.csv")
shifting_gaps_valid_csv_path = os.path.join(shifting_gaps_dir,"shifting_gaps_validation.csv")
shifting_gaps_train_data = pd.read_csv(shifting_gaps_train_csv_path)
shifting_gaps_test_data = pd.read_csv(shifting_gaps_test_csv_path)
shifting_gaps_valid_data = pd.read_csv(shifting_gaps_valid_csv_path)

### Train Data

In [ ]:
# X - Map image file
# Y - Path image file

x_train_imgs = glob.glob(os.path.join(shifting_gaps_dir, "train/*.png"))
x_imgs_train = []
x_labels_train_shifting_gaps = []
for img_path in x_train_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_train_shifting_gaps.append(img_name)
print("X_train_shifting_gaps")
print(f"# of images: {len(x_imgs_train)}")
X_train_shifting_gaps = np.array(x_imgs_train, dtype='float32')
X_train_shifting_gaps/=255
print(f"{X_train_shifting_gaps.ndim}")
print(f"{X_train_shifting_gaps.shape}")
# print(f"{X_train_shifting_gaps.dtype.names}")
# print(f"{X_train_shifting_gaps[0]}")

y_train_imgs = glob.glob(os.path.join(shifting_gaps_dir, "train_path/*.png"))
y_imgs_train = []
y_labels_train_shifting_gaps = []
for img_path in y_train_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_train_shifting_gaps.append(img_name)
print("y_train_shifting_gaps")
print(f"# of images: {len(y_imgs_train)}")
y_train_shifting_gaps = np.array(y_imgs_train, dtype='float32')
y_train_shifting_gaps/=255
print(f"{y_train_shifting_gaps.ndim}")
print(f"{y_train_shifting_gaps.shape}")
# print(f"{y_train_shifting_gaps.dtype.names}")
# print(f"{y_train_shifting_gaps[0]}")

### Validation Data

In [ ]:
x_valid_imgs = glob.glob(os.path.join(shifting_gaps_dir, "validation/*.png"))
x_imgs_valid = []
x_labels_valid_shifting_gaps = []
for img_path in x_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_valid_shifting_gaps.append(img_name)
print("X_valid_shifting_gaps")
print(f"# of images: {len(x_imgs_valid)}")
X_valid_shifting_gaps = np.array(x_imgs_valid, dtype='float32')
X_valid_shifting_gaps/=255
print(f"{X_valid_shifting_gaps.ndim}")
print(f"{X_valid_shifting_gaps.shape}")
print(f"{X_valid_shifting_gaps.dtype.names}")
print(f"{X_valid_shifting_gaps[0]}")

y_valid_imgs = glob.glob(os.path.join(shifting_gaps_dir, "validation_path/*.png"))
y_imgs_valid = []
y_labels_valid_shifting_gaps = []
for img_path in y_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_valid_shifting_gaps.append(img_name)
print("y_valid_shifting_gaps")
print(f"# of images: {len(y_imgs_valid)}")
y_valid_shifting_gaps = np.array(y_imgs_valid, dtype='float32')
y_valid_shifting_gaps/=255
print(f"{y_valid_shifting_gaps.ndim}")
print(f"{y_valid_shifting_gaps.shape}")
print(f"{y_valid_shifting_gaps.dtype.names}")
print(f"{y_valid_shifting_gaps[0]}")

### Test Data

In [ ]:
x_test_imgs = glob.glob(os.path.join(shifting_gaps_dir, "test/*.png"))
imgs_test = []
x_labels_test_shifting_gaps = []
for img_path in x_test_imgs:
    img = io.imread(img_path, as_gray=True)
    imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_test_shifting_gaps.append(img_name)
print("X_test_shifting_gaps")
print(f"# of images: {len(imgs_test)}")
X_test_shifting_gaps = np.array(imgs_test, dtype='float32')
X_test_shifting_gaps/=255
# X_test = np.array(imgs_test)
print(f"{X_test_shifting_gaps.ndim}")
print(f"{X_test_shifting_gaps.shape}")
print(f"{X_test_shifting_gaps.dtype.names}")
print(f"{X_test_shifting_gaps[0]}")

y_test_imgs = glob.glob(os.path.join(shifting_gaps_dir, "test_path/*.png"))
y_imgs_test = []
y_labels_test_shifting_gaps = []
for img_path in y_test_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_test_shifting_gaps.append(img_name)
print("y_test_shifting_gaps")
print(f"# of images: {len(y_imgs_test)}")
y_test_shifting_gaps = np.array(y_imgs_test, dtype='float32')
y_test_shifting_gaps/=255
print(f"{y_test_shifting_gaps.ndim}")
print(f"{y_test_shifting_gaps.shape}")
print(f"{y_test_shifting_gaps.dtype.names}")
print(f"{y_test_shifting_gaps[0]}")

In [ ]:
# input_shape = (32, 32, 1)
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    X_train_shifting_gaps = X_train_shifting_gaps.reshape(X_train_shifting_gaps.shape[0], 1, img_rows, img_cols)
    X_valid_shifting_gaps = X_valid_shifting_gaps.reshape(X_valid_shifting_gaps.shape[0], 1, img_rows, img_cols)
    X_test_shifting_gaps = X_test_shifting_gaps.reshape(X_test_shifting_gaps.shape[0], 1, img_rows, img_cols)
    y_train_shifting_gaps = y_train_shifting_gaps.reshape(y_train_shifting_gaps.shape[0], 1, img_rows, img_cols)
    y_valid_shifting_gaps = y_valid_shifting_gaps.reshape(y_valid_shifting_gaps.shape[0], 1, img_rows, img_cols)
    y_test_shifting_gaps = y_test_shifting_gaps.reshape(y_test_shifting_gaps.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train_shifting_gaps = X_train_shifting_gaps.reshape(X_train_shifting_gaps.shape[0], img_rows, img_cols, 1)
    X_valid_shifting_gaps = X_valid_shifting_gaps.reshape(X_valid_shifting_gaps.shape[0], img_rows, img_cols, 1)
    X_test_shifting_gaps = X_test_shifting_gaps.reshape(X_test_shifting_gaps.shape[0], img_rows, img_cols, 1)
    y_train_shifting_gaps = y_train_shifting_gaps.reshape(y_train_shifting_gaps.shape[0], img_rows, img_cols, 1)
    y_valid_shifting_gaps = y_valid_shifting_gaps.reshape(y_valid_shifting_gaps.shape[0], img_rows, img_cols, 1)
    y_test_shifting_gaps = y_test_shifting_gaps.reshape(y_test_shifting_gaps.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

## Forest

In [ ]:
# Forest
forest_dir = os.path.join(data_dir, "forest" )
forest_train_csv_path = os.path.join(forest_dir,"forest_train.csv")
forest_test_csv_path = os.path.join(forest_dir,"forest_test.csv")
forest_valid_csv_path = os.path.join(forest_dir,"forest_validation.csv")
forest_train_data = pd.read_csv(forest_train_csv_path)
forest_test_data = pd.read_csv(forest_test_csv_path)
forest_valid_data = pd.read_csv(forest_valid_csv_path)

### Train Data

In [ ]:
# X - Map image file
# Y - Path image file

x_train_imgs = glob.glob(os.path.join(forest_dir, "train/*.png"))
x_imgs_train = []
x_labels_train_forest = []
for img_path in x_train_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_train_forest.append(img_name)
print("X_train_forest")
print(f"# of images: {len(x_imgs_train)}")
X_train_forest = np.array(x_imgs_train, dtype='float32')
X_train_forest/=255
print(f"{X_train_forest.ndim}")
print(f"{X_train_forest.shape}")
# print(f"{X_train_forest.dtype.names}")
# print(f"{X_train_forest[0]}")

y_train_imgs = glob.glob(os.path.join(forest_dir, "train_path/*.png"))
y_imgs_train = []
y_labels_train_forest = []
for img_path in y_train_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_train.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_train_forest.append(img_name)
print("y_train_forest")
print(f"# of images: {len(y_imgs_train)}")
y_train_forest = np.array(y_imgs_train, dtype='float32')
y_train_forest/=255
print(f"{y_train_forest.ndim}")
print(f"{y_train_forest.shape}")
# print(f"{y_train_forest.dtype.names}")
# print(f"{y_train_forest[0]}")

### Validation Data

In [ ]:
x_valid_imgs = glob.glob(os.path.join(forest_dir, "validation/*.png"))
x_imgs_valid = []
x_labels_valid_forest = []
for img_path in x_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    x_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_valid_forest.append(img_name)
print("X_valid_forest")
print(f"# of images: {len(x_imgs_valid)}")
X_valid_forest = np.array(x_imgs_valid, dtype='float32')
X_valid_forest/=255
print(f"{X_valid_forest.ndim}")
print(f"{X_valid_forest.shape}")
# print(f"{X_valid_forest.dtype.names}")
# print(f"{X_valid_forest[0]}")

y_valid_imgs = glob.glob(os.path.join(forest_dir, "validation_path/*.png"))
y_imgs_valid = []
y_labels_valid_forest = []
for img_path in y_valid_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_valid.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_valid_forest.append(img_name)
print("y_valid_forest")
print(f"# of images: {len(y_imgs_valid)}")
y_valid_forest = np.array(y_imgs_valid, dtype='float32')
y_valid_forest/=255
print(f"{y_valid_forest.ndim}")
print(f"{y_valid_forest.shape}")
# print(f"{y_valid_forest.dtype.names}")
# print(f"{y_valid_forest[0]}")

### Test Data

In [ ]:
x_test_imgs = glob.glob(os.path.join(forest_dir, "test/*.png"))
imgs_test = []
x_labels_test_forest = []
for img_path in x_test_imgs:
    img = io.imread(img_path, as_gray=True)
    imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    x_labels_test_forest.append(img_name)
print("X_test_forest")
print(f"# of images: {len(imgs_test)}")
X_test_forest = np.array(imgs_test, dtype='float32')
X_test_forest/=255
# X_test = np.array(imgs_test)
print(f"{X_test_forest.ndim}")
print(f"{X_test_forest.shape}")
# print(f"{X_test_forest.dtype.names}")
# print(f"{X_test_forest[0]}")

y_test_imgs = glob.glob(os.path.join(forest_dir, "test_path/*.png"))
y_imgs_test = []
y_labels_test_forest = []
for img_path in y_test_imgs:
    img = io.imread(img_path, as_gray=True)
    y_imgs_test.append(img)
    img_name = img_path.split('/')[-1]
    img_name = img_name.split('.')[0]
    y_labels_test_forest.append(img_name)
print("y_test_forest")
print(f"# of images: {len(y_imgs_test)}")
y_test_forest = np.array(y_imgs_test, dtype='float32')
y_test_forest/=255
print(f"{y_test_forest.ndim}")
print(f"{y_test_forest.shape}")
# print(f"{y_test_forest.dtype.names}")
# print(f"{y_test_forest[0]}")

In [ ]:
# input_shape = (32, 32, 1)
img_rows, img_cols = 32, 32
if K.image_data_format() == 'channels_first':
    X_train_forest = X_train_forest.reshape(X_train_forest.shape[0], 1, img_rows, img_cols)
    X_valid_forest = X_valid_forest.reshape(X_valid_forest.shape[0], 1, img_rows, img_cols)
    X_test_forest = X_test_forest.reshape(X_test_forest.shape[0], 1, img_rows, img_cols)
    y_train_forest = y_train_forest.reshape(y_train_forest.shape[0], 1, img_rows, img_cols)
    y_valid_forest = y_valid_forest.reshape(y_valid_forest.shape[0], 1, img_rows, img_cols)
    y_test_forest = y_test_forest.reshape(y_test_forest.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train_forest = X_train_forest.reshape(X_train_forest.shape[0], img_rows, img_cols, 1)
    X_valid_forest = X_valid_forest.reshape(X_valid_forest.shape[0], img_rows, img_cols, 1)
    X_test_forest = X_test_forest.reshape(X_test_forest.shape[0], img_rows, img_cols, 1)
    y_train_forest = y_train_forest.reshape(y_train_forest.shape[0], img_rows, img_cols, 1)
    y_valid_forest = y_valid_forest.reshape(y_valid_forest.shape[0], img_rows, img_cols, 1)
    y_test_forest = y_test_forest.reshape(y_test_forest.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

## Compile Model

In [ ]:
import keras
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

# Build model
model = path_planning_model()
model.summary()

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile ( optimizer=adam,
                loss='mse',
                metrics=['accuracy'])

batch_size = 128
num_classes = 1
# epochs = 12
epochs = 100

## Maze Model


In [ ]:
# Train Model for Mazes
start_time = time.time()

model.fit(X_train_mazes, y_train_mazes,
        batch_size=batch_size,
        epochs=epochs,
        verbose=2,
        validation_data=(X_valid_mazes, y_valid_mazes))
score = model.evaluate(X_test_mazes, y_test_mazes, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

model.save("path-planning-mazes.h5")

In [ ]:
# Prediction on maze test values
pred_mazes = model.predict(X_test_mazes)
print("Predict image")
print(pred_mazes[0])
plt.imshow(pred_mazes[0], cmap='gray', interpolation='nearest')


In [ ]:
# Save prediction images
output_dir = os.path.join(root_dir,"output")
mazes_output_dir = os.path.join(output_dir,"mazes")
for i,p in enumerate(pred_mazes):
    img_name = x_labels_test_mazes[i]
    # print(img_name)
    io.imsave(os.path.join(mazes_output_dir,img_name+".png"), p)


In [ ]:
print(y_labels_test_mazes[0])
plt.imshow(X_test_mazes[0], cmap='gray', interpolation='nearest')

In [ ]:
plt.imshow(y_test_mazes[0], cmap='gray', interpolation='nearest')

In [ ]:
chart_regression(pred_mazes,y_test_mazes)

## Alternaing Gaps Model

In [ ]:
# Train Model for Alternating Gaps
start_time = time.time()

model.fit(X_train_alternating_gaps, y_train_alternating_gaps,
        batch_size=batch_size,
        epochs=epochs,
        verbose=2,
        validation_data=(X_valid_alternating_gaps, y_valid_alternating_gaps))
score = model.evaluate(X_test_alternating_gaps, y_test_alternating_gaps, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

model.save("path-planning-alternating_gaps.h5")

In [ ]:
# Prediction on maze test values
pred_alternating_gaps = model.predict(X_test_alternating_gaps)
print("Predict image")
print(pred_alternating_gaps[0])
plt.imshow(pred_alternating_gaps[0], cmap='gray', interpolation='nearest')


In [ ]:
# Save prediction images
output_dir = os.path.join(root_dir,"output")
alternating_gaps_output_dir = os.path.join(output_dir,"alternating_gaps")
for i,p in enumerate(pred_alternating_gaps):
    img_name = x_labels_test_alternating_gaps[i]
    # print(img_name)
    io.imsave(os.path.join(alternating_gaps_output_dir,img_name+".png"), p)


In [ ]:
print(y_labels_test_alternating_gaps[0])
plt.imshow(X_test_alternating_gaps[0], cmap='gray', interpolation='nearest')

In [ ]:
plt.imshow(y_test_alternating_gaps[0], cmap='gray', interpolation='nearest')

In [ ]:
chart_regression(pred_alternating_gaps,y_test_alternating_gaps)

## Bugtrap Forest Model

In [ ]:
# Train Model for Bugtrap Forest
start_time = time.time()

model.fit(X_train_bugtrap_forest, y_train_bugtrap_forest,
        batch_size=batch_size,
        epochs=epochs,
        verbose=2,
        validation_data=(X_valid_bugtrap_forest, y_valid_bugtrap_forest))
score = model.evaluate(X_test_bugtrap_forest, y_test_bugtrap_forest, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

model.save("path-planning-bugtrap_forest.h5")

In [ ]:
# Prediction on maze test values
pred_bugtrap_forest = model.predict(X_test_bugtrap_forest)
print("Predict image")
print(pred_bugtrap_forest[0])
plt.imshow(pred_bugtrap_forest[0], cmap='gray', interpolation='nearest')


In [ ]:
# Save prediction images
output_dir = os.path.join(root_dir,"output")
bugtrap_forest_output_dir = os.path.join(output_dir,"bugtrap_forest")
for i,p in enumerate(pred_bugtrap_forest):
    img_name = x_labels_test_bugtrap_forest[i]
    # print(img_name)
    io.imsave(os.path.join(bugtrap_forest_output_dir,img_name+".png"), p)


In [ ]:
print(y_labels_test_bugtrap_forest[0])
plt.imshow(X_test_bugtrap_forest[0], cmap='gray', interpolation='nearest')

In [ ]:
plt.imshow(y_test_bugtrap_forest[0], cmap='gray', interpolation='nearest')

In [ ]:
chart_regression(pred_bugtrap_forest,y_test_bugtrap_forest)

## Multiple Bugtraps Model

In [ ]:
# Train Model for Mazes
start_time = time.time()

model.fit(X_train_multiple_bugtraps, y_train_multiple_bugtraps,
        batch_size=batch_size,
        epochs=epochs,
        verbose=2,
        validation_data=(X_valid_multiple_bugtraps, y_valid_multiple_bugtraps))
score = model.evaluate(X_test_multiple_bugtraps, y_test_multiple_bugtraps, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

model.save("path-planning-multiple_bugtraps.h5")

In [ ]:
# Prediction on maze test values
pred_multiple_bugtraps = model.predict(X_test_multiple_bugtraps)
print("Predict image")
print(pred_multiple_bugtraps[0])
plt.imshow(pred_multiple_bugtraps[0], cmap='gray', interpolation='nearest')


In [ ]:
# Save prediction images
output_dir = os.path.join(root_dir,"output")
multiple_bugtraps_output_dir = os.path.join(output_dir,"multiple_bugtraps")
for i,p in enumerate(pred_multiple_bugtraps):
    img_name = x_labels_test_multiple_bugtraps[i]
    # print(img_name)
    io.imsave(os.path.join(multiple_bugtraps_output_dir,img_name+".png"), p)


In [ ]:
print(y_labels_test_multiple_bugtraps[0])
plt.imshow(X_test_multiple_bugtraps[0], cmap='gray', interpolation='nearest')

In [ ]:
plt.imshow(y_test_multiple_bugtraps[0], cmap='gray', interpolation='nearest')

In [ ]:
chart_regression(pred_multiple_bugtraps,y_test_multiple_bugtraps)

## Shifting Gaps Model

In [ ]:
# Train Model for Mazes
start_time = time.time()

model.fit(X_train_shifting_gaps, y_train_shifting_gaps,
        batch_size=batch_size,
        epochs=epochs,
        verbose=2,
        validation_data=(X_valid_shifting_gaps, y_valid_shifting_gaps))
score = model.evaluate(X_test_shifting_gaps, y_test_shifting_gaps, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

model.save("path-planning-shifting_gaps.h5")

In [ ]:
# Prediction on maze test values
pred_shifting_gaps = model.predict(X_test_shifting_gaps)
print("Predict image")
print(pred_shifting_gaps[0])
plt.imshow(pred_shifting_gaps[0], cmap='gray', interpolation='nearest')


In [ ]:
# Save prediction images
output_dir = os.path.join(root_dir,"output")
shifting_gaps_output_dir = os.path.join(output_dir,"shifting_gaps")
for i,p in enumerate(pred_shifting_gaps):
    img_name = x_labels_test_shifting_gaps[i]
    # print(img_name)
    io.imsave(os.path.join(shifting_gaps_output_dir,img_name+".png"), p)


In [ ]:
print(y_labels_test_shifting_gaps[0])
plt.imshow(X_test_shifting_gaps[0], cmap='gray', interpolation='nearest')

In [ ]:
plt.imshow(y_test_shifting_gaps[0], cmap='gray', interpolation='nearest')

In [ ]:
chart_regression(pred_shifting_gaps,y_test_shifting_gaps)

## Forest Model

In [ ]:
# Train Model for Mazes
start_time = time.time()

model.fit(X_train_forest, y_train_forest,
        batch_size=batch_size,
        epochs=epochs,
        verbose=2,
        validation_data=(X_valid_forest, y_valid_forest))
score = model.evaluate(X_test_forest, y_test_forest, verbose=0)
print('Test loss: {}'.format(score[0]))
print('Test accuracy: {}'.format(score[1]))

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

model.save("path-planning-forest.h5")

In [ ]:
# Prediction on maze test values
pred_forest = model.predict(X_test_forest)
print("Predict image")
print(pred_forest[0])
plt.imshow(pred_forest[0], cmap='gray', interpolation='nearest')


In [ ]:
# Save prediction images
output_dir = os.path.join(root_dir,"output")
forest_output_dir = os.path.join(output_dir,"forest")
for i,p in enumerate(pred_forest):
    img_name = x_labels_test_forest[i]
    # print(img_name)
    io.imsave(os.path.join(forest_output_dir,img_name+".png"), p)


In [ ]:
print(y_labels_test_forest[0])
plt.imshow(X_test_forest[0], cmap='gray', interpolation='nearest')

In [ ]:
plt.imshow(y_test_forest[0], cmap='gray', interpolation='nearest')

In [ ]:
chart_regression(pred_forest,y_test_forest)